# データ可視化試行講座　配布用ノートブック

*matplotlib gallery: https://matplotlib.org/gallery.html

## 1. matplotlibを利用したデータ可視化の基礎

In [ ]:
# matplotlibのインポート

In [ ]:
# 簡単なプロット

In [ ]:
# サブプロット

In [ ]:
# ループを利用したサブプロットの作成

In [ ]:
# プロット間のスペースの調整

fig, axes = plt.subplots(2,3,sharex=True,sharey=True)
for i in range(2):
    for j in range(3):
        axes[i,j].plot(np.random.randn(10))
        
plt.subplots_adjust(wspace=0,hspace=0)

## 2. Pima indiansデータのロード、データの前処理

In [ ]:
# e-learningで学習した内容ですので、以下をコピペしてください
import pandas as pd

# Jupyter Notebookのホームディレクトリに下記csvファイルを格納しておく必要があります
filename = "pima-indians-diabetes.data.csv"

df = pd.read_csv(filename, sep=',',
                 names=[
                     'Pregnancies',
                     'Glucose',
                     'Blood Pressure',
                     'Skin Thickness',
                     'Insulin',
                     'BMI',
                     'Diabetes Pedigree Function',
                     'Age',
                     'Outcome'
                 ]
                )
df.head()

In [ ]:
# 平均値による欠損値の補完 (e-learningの復習)。ここもコピペしてください。
imputer_cols = df.columns[[1,2,3,4,5,6]]
correct_df = df.copy()

for i in imputer_cols:
    correct_df[i] = correct_df[i].mask(df[i]==0, df[i].mean())

correct_df.head()

## 3. ヒストグラムによる可視化

In [ ]:
# ageデータを使用して、ヒストグラムを作成します

In [ ]:
# ビジュアルの調整

In [ ]:
# Outcomeごとにデータを分離

In [ ]:
# ヒストグラムを重ねて表示

In [ ]:
# ヒストグラムを並べて表示するための関数を作成
# 全てタイプすると時間がかかるため、ここは以下をコピペしてください

def plot_histogram(ax,df_pos,df_neg,col='Age',le=False,ti=False):
    ax.hist([df_pos[col], df_neg[col]],edgecolor='black')
    ax.set_xlabel(col)
    ax.set_ylabel('count')
    if le:
        ax.legend(['positive','negative'])
    if ti:
        s = 'Histogram of ' + col + ' data'
        ax.set_title(s)

In [ ]:
# ヒストグラムを並べて表示

## 4. 円グラフによる可視化

In [ ]:
# BMIデータを分割し、カウントします

In [ ]:
# 円グラフをプロットします

## 5. 散布図による可視化

In [ ]:
# AgeとBMIの関係をプロット

In [ ]:
# ビジュアルの調整

In [ ]:
# Outcomeごとに分けてプロットするための関数を作成します
# 全てタイプすると時間がかかるため、以下をコピペしてください

def plot_scatter(df_pos,df_neg,cols=['Age','BMI'],dsize=10):
    plt.scatter(df_pos[cols[0]],df_pos[cols[1]],
                c='orange',
                s=dsize,
                label='positive',
               )
    plt.scatter(df_neg[cols[0]],df_neg[cols[1]],
                c='blue',
                s=dsize,
                label='negative',
                alpha=0.3, # そのままだとドットが重なって見にくいため、透明度を調整
               )

    plt.xlabel(cols[0])
    plt.ylabel(cols[1])

    # legendを可視化
    plt.legend()

In [ ]:
# 作成した関数を用いてプロット

## 6. ヒートマップによる可視化

In [ ]:
# 上で描いたscatter plotをヒートマップに変換

# ヒートマップを作成するためには、2次元平面をグリッド状に分割し、
# 各領域に対して「密度」などの何らかの値を定義する必要がある。
# ここでは、単純に各領域に含まれる人数をカウント

In [ ]:
# matplotlibのimshow関数を使用

In [ ]:
# ビジュアルの調整

## 7. Logistic回帰とROC解析、ROC曲線

In [ ]:
# まずLogistic regressionを行う
# e-learningの復習となりますので、以下をコピペで先に進んでください

from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 説明変数の選択
X = correct_df[['Age','BMI']]
y = correct_df.loc[:, 'Outcome']

# 訓練データと評価データの分割
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=1/5, random_state=0
)

# インスタンスを生成します
lr = LogisticRegression(solver='lbfgs', class_weight='balanced',
                        max_iter=1e5, random_state=0)

# 学習（パラメータの推定）
# ここでは、理解を容易にするためデータを標準化せずに使用します
lr.fit(X_train, y_train)

# DataFrameに格納
coefs = pd.DataFrame(lr.coef_, columns=X.columns)
coefs['intercept'] = lr.intercept_

# DataFrameの可視化
coefs.T.plot(y=coefs.index, kind='bar', title='coefficients', grid=True)

In [ ]:
# Logistic regressionの結果を可視化するために、ageおよびBMIそれぞれの軸に対して等差数列を作成します

In [ ]:
# age x BMI の２次元平面の各座標において、regressionで求めた関数から陽性率を計算します

In [ ]:
# 陽性率の関数とデータの散布図を同一グラフ上に描画

In [ ]:
# 混合行列、classification_reportの確認
# e-learningの復習ですので、以下をコピペしてください

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

y_pred = lr.predict(X_test)
labels = [1,0]
print(classification_report(y_test, y_pred, labels=labels))

# 混合行列を出力します
# 医療検査で1を陽性、0を陰性とすると、左上は真陽性(true positive)、右下は真陰性(true negative)、
# 右上は偽陽性(false positive)、左下は偽陰性(false negative)になります。
conf_mtx = confusion_matrix(y_test, y_pred, labels=labels)
sns.heatmap(conf_mtx, annot=True, xticklabels=labels, yticklabels=labels)
plt.show()

In [ ]:
# ROC curve, およびarea under the curve (AUC) の計算

In [ ]:
# おまけ
# ROC curveの簡単な解説

tmp = [[y_test.iloc[i],prob_positive[i]] for i in range(y_test.shape[0])]
prob_pos = [v[1] for v in tmp if v[0]==1]
prob_neg = [v[1] for v in tmp if v[0]==0]

fig = plt.figure(figsize=[8,6])
ax = fig.add_subplot(1,1,1)
plot_scatter(df_pos,df_neg,dsize=10)
im = ax.imshow(prediction.T,
               extent=extent,
               origin='lower',
               cmap='pink'
               )
ax.set_xlabel('age')
ax.set_ylabel('BMI')
cb = fig.colorbar(im)
cb.set_label('P(outcome=1)')

plt.figure(figsize=[6,4])
bins = np.linspace(0,1,11)
plt.hist([prob_neg,prob_pos], bins=bins, density=True, edgecolor='black')
plt.xlabel('P(outcome=1)')
plt.ylabel(' ')
plt.legend(['negative','positive'])

plt.figure(figsize=[6,4])
plt.plot(fpr, tpr)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')

plt.show()

## 8. データ可視化演習

### 演習問題1

Obesity群 (BMI 30以上) とnormal群 (BMI 30未満) の比率を表す円グラフを糖尿病群 (Outcome=1) と非糖尿病群 (Outcome=0) で分けて作成し、それらを並べたfigureを作成してください。

### 演習問題2

AgeとBMIを説明変数として使用した場合のロジスティック回帰モデルでは、AUCが0.72程度でした。<br>
実は、この2変数の組み合わせは、分類精度という観点においては必ずしも最適ではありません。そこで、<br>

i.  AgeとBMIの組み合わせよりも「良さそうな」組み合わせを、今回学んだデータ可視化の手法を用いて探してみてください。<br>
ii. 見つけた変数の組み合わせを使用したロジスティック回帰モデルを作成し、AUCの値を表示したROC曲線を描いてください。